In [1]:
using Plots;
using DelimitedFiles;
using AugmentedGaussianProcesses;

In [2]:
X = readdlm("data/banana_X_train");
Y = readdlm("data/banana_Y_train")[:];

In [3]:
# Run sparse classification with increasing number of inducing points
Ms = [4, 8, 16, 32, 64]
models = Vector{AbstractGP}(undef,length(Ms)+1)
kernel = RBFKernel(1.0)
for (index, num_inducing) in enumerate(Ms)
    m = SVGP(X, Y, kernel,AugmentedLogisticLikelihood(),AnalyticInference(),num_inducing,Autotuning=true)
    println("Training with $(num_inducing) points")
    @time train!(m,iterations=20)
    models[index]=m;
end

Training with 4 points
  6.422074 seconds (30.05 M allocations: 1.600 GiB, 8.10% gc time)
Training with 8 points
  0.088087 seconds (478.40 k allocations: 154.363 MiB, 20.07% gc time)
Training with 16 points
  0.147010 seconds (478.40 k allocations: 254.029 MiB, 12.67% gc time)
Training with 32 points
  0.222070 seconds (478.40 k allocations: 454.379 MiB, 12.38% gc time)
Training with 64 points
  0.518321 seconds (478.75 k allocations: 859.182 MiB, 22.40% gc time)


In [4]:
mfull = VGP(X, Y, kernel,AugmentedLogisticLikelihood(),AnalyticInference(),Autotuning=true)
train!(mfull,iterations=5);
models[end] = mfull;

In [5]:
function compute_Grid(model,nGrid=50)
    mins = [-3.25,-2.85]
    maxs = [3.65,3.4]
    xlin = range(mins[1],stop=maxs[1],length=nGrid)
    ylin = range(mins[2],stop=maxs[2],length=nGrid)
    Xplot = hcat([j for i in xlin, j in ylin][:],[i for i in xlin, j in ylin][:])
    y =  proba_y(model,Xplot)
    return (y,xlin,ylin)
    end;

In [6]:
function plotdata(X,Y)
    plot(X[:,1],X[:,2],alpha=0.2,color=Y,t=:scatter,markerstrokewidth=0.0,lab="",size=(300,500));
    end;

In [8]:
function plotcontour(model,X,Y)
    nGrid = 40
    (predic,x,y) = compute_Grid(model,nGrid);
    plotdata(X,Y)
    if in(:Z,fieldnames(typeof(model)))
        println("Plotting model with $(model.nFeature) points")
        plot!(model.Z[1][:,1],model.inducingPoints.Z[1][:,2],msize=2.0,color="black",t=:scatter,lab="")
    end
    return plot!(x,y,reshape(predic,nGrid,nGrid),cbar=false,t=:contour,levels=[0.5],fill=false,fillalpha=0.2,title=(in(:Z,fieldnames(typeof(model))) ? "M = $(model.nFeature)" : "full"))
end;

In [7]:
plot(broadcast(x->plotcontour(x,X,Y),models)...,layout=(1,length(models)),size=(1000,200))

UndefVarError: UndefVarError: plotcontour not defined